In [1]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, MultiSelect, Button, ColumnDataSource, DateRangeSlider, CheckboxButtonGroup, MultiChoice
from bokeh.events import ButtonClick, MenuItemClick
from bokeh.layouts import row, column
output_notebook()

Welcome to JupyROOT 6.24/06


Loading BokehJS ...

In [2]:
CCDB = pd.read_csv("../../Data/UserFiles/CCDB.csv",header=0, sep = r',', skipinitialspace = True)
files = list(set(CCDB.Name))
paths = list(set(CCDB.Path))
time_stamps = list(set(CCDB.TimeStamp))
types = list(set(CCDB.Type))
tasks = list(set(CCDB.Task))

In [3]:
files_data = CCDB.copy(deep=True)

selected_files = files_data.copy(deep=True)
shown_files = files_data.copy(deep=True)

selected_data = []
selected_types = []
selected_tasks = []
time_range = [files_data['TimeStamp'].min(),files_data['TimeStamp'].max()]

In [7]:
def file_browser(doc):       
    def update_button_label():
        global selected_files
        label = "Download {} files".format(str(len(selected_files.index)))
        button.label = label
        
    def update_file_select_list():
        global selected_files, time_range, selected_types, selected_tasks, files_data, shown_files
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        shown_files = files_data[type_condition & time_condition & task_condition]
        file_select.options = list(set(shown_files['Name']))
        #file_select.value = []
        
    def update_time_range():
        global shown_files
        if not shown_files.empty:
            date_range.value=[shown_files['TimeStamp'].min(),shown_files['TimeStamp'].max()]
            date_range.start=shown_files['TimeStamp'].min()
            date_range.end=shown_files['TimeStamp'].max()
    
    def update_type_list():
        global selected_tasks, time_range, files_data
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        existing_types = files_data[time_condition & task_condition]
        type_select.options = list(set(existing_types['Type']))
        
    def update_selection():
        global selected_data, selected_files, time_range, selected_types, selected_tasks, files_data
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        name_condition = files_data['Name'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        selected_files = files_data[name_condition & time_condition & type_condition & task_condition]
    
    def select_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, selected_tasks
        selected_data = file_select.value
        update_selection()
        update_button_label()
        
    def date_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        time_range = date_range.value
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def type_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        selected_types = type_select.value
        update_selection()
        update_file_select_list()
        update_time_range()
        update_button_label()
        
    def task_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        selected_tasks = task_select.value
        update_selection()
        update_file_select_list()
        update_time_range()
        update_button_label()
        update_type_list()
        
    def button_callback(event):
        global selected_data, selected_files, files_data, time_range, selected_types
        display(selected_files)

    type_select = MultiChoice(title='Object type',options=types,value=[])
    type_select.on_change("value",type_callback)
    
    task_select = MultiChoice(title='QC task',options=tasks,value=[])
    task_select.on_change("value",task_callback)
        
    date_range = DateRangeSlider(title='Time range',value=[files_data['TimeStamp'].min(),files_data['TimeStamp'].max()],
                                    start=files_data['TimeStamp'].min(), end=files_data['TimeStamp'].max())
    date_range.on_change("value_throttled",date_callback)    
    
    file_select = MultiSelect(title="Files", value=[], options=files, height=300)
    file_select.on_change("value", select_callback)

    button = Button(label="Download all", button_type="success") 
    button.on_event(ButtonClick, button_callback)
    
    doc.add_root(row(column(task_select,type_select,date_range,button),file_select))
show(file_browser)

,ID,Path,Name,TimeStamp,Type,Task
38442,38443,qc/TPC/MO/PadCalibration/,c_ROCs_Pedestals_1D,1612962508645,TCanvas,PadCalibration
38443,38444,qc/TPC/MO/PadCalibration/,c_ROCs_Pedestals_1D,1612976187111,TCanvas,PadCalibration
38457,38458,qc/TPC/MO/PadCalibration/,c_ROCs_Pedestals_1D,1613026671600,TCanvas,PadCalibration


In [126]:
display(selected_files)

,ID,Path,Name,TimeStamp,Type,Task
496,497,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1624716412346,TCanvas,Calibration
502,503,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1633360999468,TCanvas,Calibration
504,505,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1633351456892,TCanvas,Calibration
506,507,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1624628432630,TCanvas,Calibration
507,508,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1625745700348,TCanvas,Calibration
529,530,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1633367779045,TCanvas,Calibration
542,543,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1625750150505,TCanvas,Calibration
545,546,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1624704237571,TCanvas,Calibration
550,551,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1626162770208,TCanvas,Calibration
571,572,qc/TPC/MO/Calibration/,c_ROCs_PulserT0_2D,1624636707310,TCanvas,Calibration
